# 🧹 02_Limpieza de Datos



En este notebook realizamos las tareas de limpieza necesarias para preparar los datasets 
**bank** y **customers** para el análisis exploratorio y modelos posteriores.

## Importación de librerías
Cargamos las librerías necesarias:  
- **pandas**: para manipulación de datos  
- **numpy**: para valores nulos y operaciones numéricas  
- **datetime**: para trabajar con variables de fecha

In [17]:
# ==============================
# Paso 0: Importación de librerías
# ==============================
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Mostrar todas las columnas en pantalla
pd.set_option("display.max_columns", None)


## Carga de datos
Importamos los archivos desde la carpeta `data/Archivos_origen`.  
- `customer-details.xlsx` contiene información demográfica y de comportamiento de clientes.  
- `bank-additional.csv` contiene información de campañas de marketing bancario.  

Usamos la primera columna como índice para eliminar `Unnamed: 0`.

In [18]:

ruta_customer = r"C:\Users\HUGO\Desktop\Data Analyst\09_PYTHON\Proyecto_Python_Data\data\Archivos_origen\customer-details.xlsx"
ruta_bank = r"C:\Users\HUGO\Desktop\Data Analyst\09_PYTHON\Proyecto_Python_Data\data\Archivos_origen\bank-additional.csv"

df_customer = pd.read_excel(ruta_customer, index_col=0)
df_bank = pd.read_csv(ruta_bank, index_col=0, encoding="utf-8")

print("Datasets cargados correctamente ✅")
print("Dimensiones df_customer:", df_customer.shape)
print("Dimensiones df_bank:", df_bank.shape)


Datasets cargados correctamente ✅
Dimensiones df_customer: (20115, 6)
Dimensiones df_bank: (43000, 23)


## Tratamiento de valores 'unknown'
En el dataset `bank` muchas variables categóricas contienen la categoría `unknown`.
Estas representan datos faltantes. Vamos a reemplazarlas por `NaN` para identificarlas 
y poder decidir si imputarlas o tratarlas como categoría propia.

 


In [19]:
df_bank.replace("unknown", np.nan, inplace=True)



## Eliminación de columnas irrelevantes
En el dataset `customers`, la columna `id` podría ser solo un identificador y no aporta información.
Para evitar errores en caso de que no exista, usamos `errors="ignore"`.



In [20]:
df_customer = df_customer.drop(columns=["id"], errors="ignore")



## Variables derivadas de fechas
De la columna `dt_customer` en `customers` extraemos:
- Año de alta
- Mes de alta
- Antigüedad en días
Esto nos permitirá analizar patrones temporales en la base de clientes.


In [21]:
if "dt_customer" in df_customer.columns:
    hoy = datetime.today()
    df_customer["anio_alta"] = df_customer["dt_customer"].dt.year
    df_customer["mes_alta"] = df_customer["dt_customer"].dt.month
    df_customer["antiguedad_dias"] = (hoy - df_customer["dt_customer"]).dt.days


## Detección y tratamiento de outliers
En variables numéricas como `income` (customers) o `duration` (bank), 
existen valores extremos que pueden distorsionar el análisis.
Utilizamos el rango intercuartílico (IQR) para detectarlos y marcarlos.


In [22]:
def marcar_outliers(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    return df[columna].apply(lambda x: x < limite_inferior or x > limite_superior)

# Si existen estas columnas, marcamos outliers
if "income" in df_customer.columns:
    df_customer["outlier_income"] = marcar_outliers(df_customer, "income")

if "duration" in df_bank.columns:
    df_bank["outlier_duration"] = marcar_outliers(df_bank, "duration")


## Normalización de texto en categóricas
En `bank`, homogenizamos las variables categóricas para evitar inconsistencias:
- Pasamos todo a minúsculas.
- Eliminamos espacios extra.


In [23]:
for col in df_bank.select_dtypes(include="object").columns:
    df_bank[col] = df_bank[col].astype(str).str.strip().str.lower()
